<a href="https://colab.research.google.com/github/rahulrajpr/AdvancedHyperParameterOptimisation/blob/HyperOpt/HyperOpt_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hyperopt Optimization**

There are 3 Techniques

- Random Search - random.suggest
- Anneling - anneal.suggest
- Tree Structured Parzen Estimators - tpe.suggest

Steps

  1.Define the model

  2.Define the hyperparameter space
  
  3.Define Objective Function
  
  4.Run the minimization

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
dataset = load_breast_cancer()
data = dataset.data
target = dataset.target
columns = dataset.feature_names
df = pd.DataFrame(data = data, columns = columns)
df['target'] = target
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [4]:
X = df.drop('target', axis =1)
y = df['target']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.7, random_state = 100, stratify = y)

Parameters

(max_depth: int = 3, learning_rate: float = 0.1, n_estimators: int = 100, verbosity: int = 1, silent: Any | None = None, objective: str = "binary:logistic", booster: str = 'gbtree', n_jobs: int = 1, nthread: Any | None = None, gamma: int = 0, min_child_weight: int = 1, max_delta_step: int = 0, subsample: int = 1, colsample_bytree: int = 1, colsample_bylevel: int = 1, colsample_bynode: int = 1, reg_alpha: int = 0, reg_lambda: int = 1, scale_pos_weight: int = 1, base_score: float = 0.5, random_state: int = 0, seed: Any | None = None, missing: Any | None = None, **kwargs: Any) -> None

Define the parameter space

In [12]:
from hyperopt import hp

In [14]:
param_grid = {
              'n_estimators' : hp.quniform('n_estimators',200,2500,100),
              'max_depth' : hp.quniform('max_depth',1,10,1),
              'learning_rate':hp.loguniform('learning_rate',np.log(0.01), np.log(1)),
              'booster': hp.choice('booster',['gbtree','dart']),
              'gamma':hp.loguniform('gamma', np.log(0.1), np.log(10)),
              'colsample_bytree': hp.uniform('colsample_bytree', 0.50, 0.99),
              'colsample_bylevel': hp.uniform('colsample_bylevel', 0.50, 0.99),
              'colsample_bynode': hp.uniform('colsample_bynode', 0.50, 0.99),
              'reg_lambda': hp.uniform('reg_lambda', 1, 20)
              }

Define the objective Function

In [15]:
from sklearn.model_selection import cross_val_score

In [8]:
from xgboost import XGBClassifier

In [16]:
def objective(params):

  param_dict = {
              'n_estimators' : int(params['n_estimators']),
              'max_depth' : int(params['max_depth']),
              'learning_rate':params['learning_rate'],
              'booster': params['booster'],
              'gamma': params['gamma'],
              'colsample_bytree': params['colsample_bytree'],
              'colsample_bylevel': params['colsample_bylevel'],
              'colsample_bynode': params['colsample_bynode'],
              'reg_lambda': params['reg_lambda'],
              'random_state' : 100
              }

  xgb = XGBClassifier(**param_dict)

  scores = cross_val_score(estimator = xgb,
                           X = X_train,
                           y = y_train,
                           cv = 5,
                           n_jobs = -1,
                           scoring = 'accuracy'
                           )
  mean = np.mean(scores)

  return -mean

Define and run the search

Random Search

In [18]:
from hyperopt import fmin
from hyperopt import rand

In [19]:
random_search = fmin(fn = objective,
                     space = param_grid,
                     algo = rand.suggest,
                     max_evals = 50,
                     rstate = np.random.RandomState(100))

%time

100%|██████████| 50/50 [06:25<00:00,  7.72s/it, best loss: -0.9622784810126582]
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


In [21]:
type(random_search)

dict

In [23]:
# best hyperparameter
random_search

{'booster': 1,
 'colsample_bylevel': 0.6680605602400378,
 'colsample_bynode': 0.62489342402988,
 'colsample_bytree': 0.8119135935808236,
 'gamma': 0.1305315787332366,
 'learning_rate': 0.33155426774231894,
 'max_depth': 6.0,
 'n_estimators': 1100.0,
 'reg_lambda': 5.105515888845169}

Define a function to rearrange the hyperparameter dict

In [34]:
def rearrange(search,best_booster):

  search['booster'] = best_booster
  search['n_estimators'] = int(search['n_estimators'])
  search['max_depth'] = int(search['max_depth'])

In [48]:
rearrange(random_search,'dart')
random_search

{'booster': 'dart',
 'colsample_bylevel': 0.6680605602400378,
 'colsample_bynode': 0.62489342402988,
 'colsample_bytree': 0.8119135935808236,
 'gamma': 0.1305315787332366,
 'learning_rate': 0.33155426774231894,
 'max_depth': 6,
 'n_estimators': 1100,
 'reg_lambda': 5.105515888845169}

lets train the model on the best hyperparameters by random search

In [49]:
xgb_random = XGBClassifier(**random_search)

In [50]:
xgb_random.fit(X_train,y_train)

XGBClassifier(booster='dart', colsample_bylevel=0.6680605602400378,
              colsample_bynode=0.62489342402988,
              colsample_bytree=0.8119135935808236, gamma=0.1305315787332366,
              learning_rate=0.33155426774231894, max_depth=6, n_estimators=1100,
              reg_lambda=5.105515888845169)

lets evaluate the model

In [51]:
from sklearn.metrics import accuracy_score

def show_metric(model):
  print('accuracy for train set : ', accuracy_score(y_train,model.predict(X_train)))
  print('accuracy for test set : ', accuracy_score(y_test,model.predict(X_test)))

In [52]:
show_metric(xgb_random)

accuracy for train set :  1.0
accuracy for test set :  0.9766081871345029


Annealing Search

In [43]:
from hyperopt import anneal

In [46]:
search_anneal = fmin(fn = objective,
                     space = param_grid,
                     algo = anneal.suggest,
                     max_evals = 50,
                     rstate = np.random.RandomState(100),
                     verbose = 1)

100%|██████████| 50/50 [05:01<00:00,  6.03s/it, best loss: -0.9623101265822784]


In [47]:
search_anneal

{'booster': 0,
 'colsample_bylevel': 0.8997506945402796,
 'colsample_bynode': 0.6685251117042704,
 'colsample_bytree': 0.8749904784962741,
 'gamma': 1.915214351334952,
 'learning_rate': 0.31326773019698684,
 'max_depth': 8.0,
 'n_estimators': 1000.0,
 'reg_lambda': 12.803041540679398}

In [54]:
rearrange(search_anneal,'gbtree')
search_anneal

{'booster': 'gbtree',
 'colsample_bylevel': 0.8997506945402796,
 'colsample_bynode': 0.6685251117042704,
 'colsample_bytree': 0.8749904784962741,
 'gamma': 1.915214351334952,
 'learning_rate': 0.31326773019698684,
 'max_depth': 8,
 'n_estimators': 1000,
 'reg_lambda': 12.803041540679398}

In [55]:
xgb_anneal = XGBClassifier(**search_anneal)

In [56]:
xgb_anneal.fit(X_train,y_train)

XGBClassifier(colsample_bylevel=0.8997506945402796,
              colsample_bynode=0.6685251117042704,
              colsample_bytree=0.8749904784962741, gamma=1.915214351334952,
              learning_rate=0.31326773019698684, max_depth=8, n_estimators=1000,
              reg_lambda=12.803041540679398)

In [57]:
show_metric(xgb_anneal)

accuracy for train set :  0.992462311557789
accuracy for test set :  0.9707602339181286


Tree Structured Parzen Estimators

In [58]:
from hyperopt import tpe

In [59]:
search_tpe = fmin(fn = objective, 
                  space = param_grid,
                  algo = tpe.suggest,
                  max_evals = 50,
                  verbose = 1)
%time

100%|██████████| 50/50 [07:53<00:00,  9.46s/it, best loss: -0.9673101265822783]
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [60]:
search_tpe

{'booster': 1,
 'colsample_bylevel': 0.867223424087598,
 'colsample_bynode': 0.6395182152431079,
 'colsample_bytree': 0.7221110949340002,
 'gamma': 2.0341073029020844,
 'learning_rate': 0.31746861989549885,
 'max_depth': 3.0,
 'n_estimators': 700.0,
 'reg_lambda': 3.0458324371626797}

In [61]:
rearrange(search_tpe,'dart')
search_tpe

{'booster': 'dart',
 'colsample_bylevel': 0.867223424087598,
 'colsample_bynode': 0.6395182152431079,
 'colsample_bytree': 0.7221110949340002,
 'gamma': 2.0341073029020844,
 'learning_rate': 0.31746861989549885,
 'max_depth': 3,
 'n_estimators': 700,
 'reg_lambda': 3.0458324371626797}

In [63]:
xgb_tpe = XGBClassifier(**search_tpe)

In [64]:
xgb_tpe.fit(X_train,y_train)

XGBClassifier(booster='dart', colsample_bylevel=0.867223424087598,
              colsample_bynode=0.6395182152431079,
              colsample_bytree=0.7221110949340002, gamma=2.0341073029020844,
              learning_rate=0.31746861989549885, n_estimators=700,
              reg_lambda=3.0458324371626797)

In [65]:
show_metric(xgb_tpe)

accuracy for train set :  0.992462311557789
accuracy for test set :  0.9707602339181286
